# Decsion Trees are based on three points named Entropy , information gain and giny impurity
# Ideal selection of nodes and predicting separate classes requires low entropy , high information gain , and low genie impurity

# Entropy is the measure of randomness in the data with the formula H(S) = - sum(p_i * log2(p_i))
# information gain computes the diffrence between the entropy after split and and average entropy after split(low entropy high information gain
# and high entropy low information gain) IG(S, A) = H(S) - sum(|S_v|/|S| * H(S_v))
 # where IG(S, A) is the information gain of attribute A on set S, H(S) is the entropy of set S, |S_v| is the number of examples in subset S_v that have value v for attribute A, and |S| is the total number of examples in set S.

# The summation is taken over all possible values of attribute A. The attribute that maximizes information gain is chosen as the splitting attribute at the current node.

# To build a decision tree, the data is recursively split based on the attributes that provide the most information gain until a stopping criterion is met, such as all examples in a node belonging to the same class or the tree reaches a certain depth.

# Information gain is a key concept in decision trees and is used to construct decision trees that are able to make accurate predictions for new, unseen data

In [96]:
# installing the dependencies

import pandas as pd
import numpy as np
from pprint import pprint
from collections import Counter
from sklearn.model_selection import train_test_split


In [97]:
# importing the data set
df = pd.read_csv("zoo.csv",names=['Animal_name','air','Feathers','Eggs','Milk','Airborne','Aquatic','Predator','Toothed','Backbone'
,'Breathes','Venomous','Fins','Legs','Tail','Domestic','Catsize','Class_type'])




In [98]:
df.drop(index=0 ,axis= 0,inplace= True)


In [99]:
df.drop('Animal_name',axis=1,inplace= True)  # the animal name is dropped as it irrelevant

In [100]:
df

,air,Feathers,Eggs,Milk,Airborne,Aquatic,Predator,Toothed,Backbone,Breathes,Venomous,Fins,Legs,Tail,Domestic,Catsize,Class_type
1,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
2,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
3,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
4,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
5,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
98,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6
99,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
100,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,7


# designing the decision tree

# Train algorithm == build the tree
1> start at the top node and at each node select the best split based on the nformation given
2>Greedy search loop all over th features and over all thresholds
3> save the best split feature and split threshold at each node
4> build the tree recursively
5> apply stopping criteria
maxmum depth , mimun samples at each node , no more class disttibution(entropy becomes zero)
6> when we have a leaf node store the most common class



# Predict ==== Traverse the tree
1> Traverse the tree recursivly
2> at each node we look at the best split feature vetorx and go left or right depending on x[feature_idx] <= threshold>

3> when we reach the leaf node we return the store the most common class label

In [101]:
# vector y siginfies all the class labels  (entropy is  E=-Σ⋅p(x)⋅log_2⁡(p(x))
# p(x) = #x/n  (x is the count of instances) = n - number of occurences

def entropy(y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])


class Node:
    def __init__(
        self, feature=None, threshold=None, left=None, right=None, *, value=None
    ):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria
        if (
            depth >= self.max_depth
            or n_labels == 1
            or n_samples < self.min_samples_split
        ):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedily select the best split according to information gain
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)

        # grow the children that result from the split
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # information gain is difference in loss before vs. after split
        ig = parent_entropy - child_entropy
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common



In [102]:
from sklearn import datasets
from sklearn.model_selection import train_test_split


data = datasets.load_breast_cancer()
X, y = data.data , data.target

X_train ,X_test ,y_train , y_test = train_test_split(X,y,test_size=0.2,random_state= 1234)

classifier = DecisionTree(max_depth=10)
classifier.fit(X_train,y_train)

In [103]:
def accuracy (a,b):
    acc = (np.sum(a==b)/len(a)) * 100

    return acc

In [104]:
y_pred = classifier.predict(X_test)
accuracy = accuracy(y_test,y_pred) 
print("Accuracy:",accuracy)

Accuracy: 92.98245614035088
